In [4]:
import os
import pandas as pd
import json
import numpy as np
#!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
#from ipynb.fs.full.my_functions import factorial

In [6]:
import tweepy
import traceback

In [9]:
import spacy
import torch

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
#!pip install spacymoji
from spacymoji import Emoji

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

#!pip install transformers
from transformers import AutoModel, BertTokenizerFast

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score

from sklearn.utils import resample
from sklearn.model_selection import GroupShuffleSplit

In [ ]:
# Get features for TF-IDF
tfidf = TfidfVectorizer()#tokenizer=spacy_tokeniser)
tfidf.fit(train_text)
train_features = tfidf.transform(train_text)
validation_features = tfidf.transform(val_text)
test_features = tfidf.transform(test_text)

In [ ]:
def evaluation_summary(description, true_labels, predictions):
  print("Evaluation for: " + description)
  print(classification_report(true_labels, predictions,  digits=3, zero_division=0))
  print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions))

In [ ]:
# SVC
svc = SVC(kernel='rbf')
svc_model = svc.fit(train_features, train_labels)
svc_test = svc_model.predict(test_features)
evaluation_summary("SVC test", test_labels, svc_test)

# Logistic Regression with TF-IDF
lr_tfidf = LogisticRegression()
lr_model_tfidf = lr_tfidf.fit(train_features, train_labels)
lrtfidf_test = lr_model_tfidf.predict(test_features)
evaluation_summary("LR (TF-IDF) test", test_labels, lrtfidf_test)

# Dummy Majority
dumb = DummyClassifier(strategy='most_frequent')
dumb.fit(train_features, train_labels)
dumb_test = dumb.predict(test_features)
evaluation_summary("Dummy MF test", test_labels, dumb_test)

0.9798766634209672
Evaluation for: SVC
              precision    recall  f1-score   support

       False      0.973     0.986     0.979     16263
        True      0.987     0.974     0.981     17628

    accuracy                          0.980     33891
   macro avg      0.980     0.980     0.980     33891
weighted avg      0.980     0.980     0.980     33891


Confusion matrix:
 [[16033   230]
 [  452 17176]]
Evaluation for: SVC test
              precision    recall  f1-score   support

       False      0.974     0.986     0.980     16264
        True      0.987     0.975     0.981     17627

    accuracy                          0.980     33891
   macro avg      0.980     0.981     0.980     33891
weighted avg      0.981     0.980     0.981     33891


Confusion matrix:
 [[16039   225]
 [  436 17191]]
0.9763359003865333
0.9760408368003305
Evaluation for: LR (TF-IDF)
              precision    recall  f1-score   support

       False      0.969     0.982     0.976     16263
     

## Make Prediction

In [ ]:
# Get output

input_text = "Watch video showing gunfire inside Canada's pa"
input_df = pd.DataFrame({"src_text": [input_text]})
input_features = tfidf.transform(input_df)

predicted_label = lr_model_tfidf.predict(input_features)

print(predicted_label)

['0']


# Import and Fix Data

In [16]:
# Combine dataset classes
real_df = pd.read_csv("data/FakeNewsNet/dataset/pf_real.csv")
fake_df = pd.read_csv("data/FakeNewsNet/dataset/pf_fake.csv")
fake_df['true'] = False
real_df['true'] = True
df = pd.concat([fake_df, real_df])

# Fix imbalance
# https://elitedatascience.com/imbalanced-classes
# Separate majority and minority classes
df_majority = df[df.true==True]
df_minority = df[df.true==False]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=len(df_minority),     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# Split train-test
# https://stackoverflow.com/questions/61337373/split-on-train-and-test-separating-by-group

# Update df to the downsampled one

X = df_downsampled.drop(labels='true', axis=1)
y = df_downsampled.true


# Split in training/testing data but in a way that Tweets referring to the same article don't appear in both datasets
gs = GroupShuffleSplit(n_splits=2, test_size=.6, random_state=0)
train_ix, test_ix = next(gs.split(X, y, groups=X.article_id))

print(type(train_ix))

X_train = X.iloc[train_ix]
y_train = y.iloc[train_ix]

X_test = X.iloc[test_ix]
y_test = y.iloc[test_ix]

<class 'numpy.ndarray'>


In [17]:
# Get features for TF-IDF

train_text = X_train['text']
train_labels = y_train

test_text = X_test['text']
test_labels = y_test

# BERT

In [18]:
# https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb#scrollTo=URn-DWJt5xhP
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

import transformers as ppb
model_class, tokenizer_class, bert_model_name = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(bert_model_name)
model = model_class.from_pretrained(bert_model_name)

C:\Users\karli\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\karli\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.pred

In [19]:
tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [20]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [22]:
print(padded)

[[  101  4911  1024 ...     0     0     0]
 [  101  4911  1024 ...     0     0     0]
 [  101  4911  1024 ...     0     0     0]
 ...
 [  101 21907  2289 ...     0     0     0]
 [  101  4931  3071 ...     0     0     0]
 [  101  1030  3565 ...     0     0     0]]


In [25]:
input_ids = torch.tensor(np.array(padded))

with torch.no_grad():
    last_hidden_states = model(input_ids) # ERROR: Tries to get 255 GB of RAM

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 485344604160 bytes.

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = df['true']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
lr_clf.score(test_features, test_labels)

0.5866666666666667

# Cleaning dataset

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
from collections import Counter

ps = PorterStemmer()
wnl = nltk.stem.WordNetLemmatizer()

stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karli\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\karli\AppData\Roaming\nltk_data...


In [ ]:
remove_col = ['threadid','tweetid','support', 'evidentiality', 'certainty', 'event']

# Impute null values with None
def null_process(feature_df):
    col = 'src_text'
    feature_df.loc[feature_df[col].isnull(), col] = "None"
    return feature_df

# Removed unused clumns
def remove_unused_col(df,column_n=remove_col):
    df = df.drop(column_n,axis=1)
    return df

def clean_dataset(df):
    # remove unused column
    df = remove_unused_col(df)
    #impute null values
    df = null_process(df)
    return df

# Cleaning text from unused characters
def clean_text(text):
    text = str(text).replace(r'http[\w:/\.]+', ' ')  # removing urls
    text = str(text).replace(r'[^\.\w\s]', ' ')  # remove everything but characters and punctuation
    text = str(text).replace('[^a-zA-Z]', ' ')
    text = str(text).replace(r'\s\s+', ' ')
    text = text.lower().strip()
    #text = ' '.join(text)    
    return text

## Nltk Preprocessing includes:
# Stop words, Stemming and Lemmetization
def nltk_preprocess(text):
    text = clean_text(text)
    wordlist = re.sub(r'[^\w\s]', '', text).split()
    text = ' '.join([word for word in wordlist if word not in stopwords_dict])
    text = [ps.stem(word) for word in wordlist if not word in stopwords_dict]
    text = ' '.join([wnl.lemmatize(word) for word in wordlist if word not in stopwords_dict])
    return  text

In [ ]:
df = clean_dataset(df)
df["src_text"] = df.src_text.apply(nltk_preprocess)
df.head()

,true,src_text
0,unverified,vladimir putin netralized internal coup maybe ...
1,unverified,coup rt jimgeraghty rumor russian military att...
2,unverified,hoppla l0gg0l swiss rumor putin absence due gi...
3,unverified,putin reappears tv amid claim unwell threat co...
4,1,france 10 people dead shooting hq satirical we...


In [ ]:
def remove_date(row):
    return row['Created at'].timetz()

tweets_df['Created at'] = tweets_df.apply(lambda row: remove_date(row), axis=1)

tweets_df.head()

,ID,Text,Created at,User location,Num Followers,Num Friends,Num Favourites,User description,User verified,Language
0,937349434668498944,BREAKING: First NFL Team Declares Bankruptcy O...,15:54:54+00:00,"Sugar Land, TX",1712,2751,22141,Ofelia. Arizmendez @ deplorable me. I am a pro...,False,en
1,937379378006282240,BREAKING: First NFL Team Declares Bankruptcy O...,17:53:54+00:00,,14,99,125,,False,en
2,937380068590055425,BREAKING: First NFL Team Declares Bankruptcy O...,17:56:38+00:00,,14,99,125,,False,en
3,937429898670600192,BREAKING: First NFL Team Declares Bankruptcy O...,21:14:39+00:00,Ozarks. Missouri,1162,1258,9422,Happily married conservative Pentecostal woman...,False,en
4,937449906352152576,BREAKING: First NFL Team Declares Bankruptcy O...,22:34:09+00:00,,12772,13708,23094,Deplorable member of The Silenced Majority.. #...,False,en


In [ ]:
tweets_df2.to_csv("final_short.csv")

hello
